In [1]:
import json
from ibm_watson import LanguageTranslatorV3
from ibm_watson import AssistantV2
from ibm_watson import TextToSpeechV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson.websocket import SynthesizeCallback
from os.path import join, dirname
import pygame
from neo4j import GraphDatabase
from pandas import DataFrame
from clips import Environment
from tkinter import ttk, messagebox
from PIL import Image, ImageTk
from tkinter import *

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


# Autenticación y credenciales de los servicios en IBM Watson

In [2]:
#ATENTICACIÓN TRANSLATE WATSON
authenticator = IAMAuthenticator('ex7fG5d2ob2SpeqnBatbjrloOjiD3fruLReO-zipIRG8')
language_translator = LanguageTranslatorV3(
    version='2018-05-01',
    authenticator=authenticator)
language_translator.set_service_url('https://api.us-south.language-translator.watson.cloud.ibm.com/instances/3f020a13-ae1f-4d67-87f2-808add2359f5')

#ATENTICACIÓN TEXT_TO_SPEECH WATSON
authenticator = IAMAuthenticator('y6g3Klma5riCxg7RWDp2UUsT9HCW07iaT2qruSdOuTwg')
text_to_speech = TextToSpeechV1( authenticator=authenticator)
text_to_speech.set_service_url('https://api.us-south.text-to-speech.watson.cloud.ibm.com/instances/6ddfab7b-3699-484e-801b-b2cbcf8cab1c')


#ATENTICACIÓN ASSISTANT WATSON
authenticator = IAMAuthenticator('Vdhmwp71ttlef0bM0JCAVGM1-jqVEt-sQlHqrWFFo_HO')
assistant = AssistantV2(
    version = '2020-04-01',
    authenticator = authenticator
)
assistant.set_service_url('https://api.us-south.assistant.watson.cloud.ibm.com/instances/d80a5eb7-c2a1-43d2-84c0-9891733fc13d')
assistant.set_disable_ssl_verification(False)

session = assistant.create_session('5d54ad00-095a-4d6f-b630-5bb1bb72fea1').get_result()

voices = text_to_speech.list_voices().get_result()
#print(json.dumps(voices, indent=2))


#AUTENTICACION NEO4J
uri = "bolt://localhost:7687"
# Connect to the neo4j database server

graphDB_Driver  = GraphDatabase.driver(uri)

## UPDATE


In [ ]:
comand_delete = 'MATCH (n) OPTIONAL MATCH (n)-[r]-() DELETE n,r RETURN count(n) as numero'

with graphDB_Driver.session() as graphDB_Session:
    
    result = graphDB_Session.run(comand_delete)
    print (result)


# INTEGRACION NEO4J

## CREATE

In [13]:
cqlCreate = """Create (h:Persona {gender:'Hombre',age:25 }),
    (m:Persona {gender:'Mujer', age:22}), 
    (q:Motivation {description:'Mejorar la educacion'}),
    (n:Motivation {description:'Vocacion de enseniar'}),
    (h)-[:RELATION {value: 3}]->(q),
    (h)-[:RELATION {value: 1}]->(n),
    (m)-[:RELATION {value: 5}]->(n)
    RETURN COUNT(h) AS personas"""

with graphDB_Driver.session() as graphDB_Session:
    
    # LISTA LOS NODOS CREADOS
    nodes = graphDB_Session.run(cqlCreate)
    
    record = nodes.single()
    print(record)
    

<Record personas=1>


## UPDATE

In [4]:
pesos=[]



def merge_mujer(desc, value):

    cql_update_query = """MERGE  (m:Persona {gender:'Mujer', age:22})
                    MERGE (q2:Motivation{description:$desc})
                    MERGE (m)-[r:RELATION]->(q2)
                    SET r += {value:$value}
                    RETURN *"""

    with graphDB_Driver.session() as graphDB_Session:

        # LISTA LOS NODOS CREADOS
        nodes = graphDB_Session.run(cql_update_query, desc=desc, value=value )
        result = nodes.keys()
        print(result)

def merge_hombre(desc, value):
    cql_update_query = """MERGE  (h:Persona {gender:'Hombre', age:25})
                    MERGE (q3:Motivation{description:$desc})
                    MERGE (h)-[r:RELATION]->(q3)
                    SET r += {value:$value}
                    RETURN *"""

    with graphDB_Driver.session() as graphDB_Session:

        # LISTA LOS NODOS CREADOS
        nodes = graphDB_Session.run(cql_update_query, desc=desc, value=value )
        result = nodes.keys()
        print(result)

## READ

In [5]:
# CQL to query all the universities present in the graph
values = []
attribute=[]
pesos=[]
def list_conditional(gender):
    cqlEdgeQuery = "match (:Persona {gender: $gender})-->(m:Motivation) return (m.description)"

    with graphDB_Driver.session() as graphDB_Session:

        nodes = graphDB_Session.run(cqlEdgeQuery, gender=gender)
        for node in nodes:
            values.append(node)
        return values
        """if gender == 'Hombre':
            desc = input("Por que escogio esta carrera? " )
            value = input("ingrese valor: ")
            merge_hombre(desc, value)"""
        
        
def values(desc, gender):
    cqlEdgeQuery = "MATCH (m)-[r:RELATION]->(q2:Motivation) where q2.description =$desc  RETURN (r.value)"
    with graphDB_Driver.session() as graphDB_Session:

        pesos = graphDB_Session.run(cqlEdgeQuery, desc=desc)
        for peso in pesos:
            prueba =str(peso)
            #prueba.split()
            primero=prueba.split("=", 2)
            segundo = str(primero[1])
            tercero = segundo.split(">",1)
            cuarto =str(tercero[0])
            print(cuarto[0])
            valor= int(cuarto[0])
            value = valor+1
            
            
        if gender == 'Hombre':
            merge_hombre(desc, value)
        if gender == 'Mujer':
            merge_mujer(desc, value)

            
            
def list_motivations():
    values=[]
    cqlEdgeQuery = "MATCH (m:Motivation) RETURN (m.description)"
    with graphDB_Driver.session() as graphDB_Session:

        nodes = graphDB_Session.run(cqlEdgeQuery)
        for node in nodes:               
            values.append(node)
            #print(node)
        return values
    
def lis_all():
    
    cqlNodeQuery = "MATCH (x) RETURN x"

    # CQL to query the distances from Yale to some of the other Ivy League universitie

    with graphDB_Driver.session() as graphDB_Session:

        # LISTA LOS NODOS CREADOS
        nodes = graphDB_Session.run(cqlNodeQuery)

        print("Nodos creados:")

        for node in nodes:

            print(node)

        #LISTAR LAS RELACIONES
                
#lis_all()
#list_conditional('Hombre')
list_motivations()

[<Record (m.description)='Vocacion de enseniar'>,
 <Record (m.description)='Mejorar la educacion'>]

# INTEGRACION CLIPS

In [7]:
env = Environment()
def expert_system(motivacion):
    env.assert_string("(motivacion "+ motivacion +")")
    #env.assert_string("(motivacion  vocacion de ayudar)")

    rule = """
    (defrule my-rule
    (motivacion ayudar)
    =>
    (printout t "deberias ingresar a un grupo" crlf))
    """

    for fact in env.facts():
        aux = str(fact)
        lower=aux.lower()
        #print(lower)
        ccc=lower.find('ayudar')
        if ccc != -1:
            saludo = env.build(rule)
            env.run()
            return('Deberias ingresar a algun grupo')
        ccc=lower.find('vocacion')
        if ccc !=-1:
            return "Tendrias que dar tutorias a tus companieros"
    
        ccc=lower.find('gusta')
        if ccc !=-1:
            return "Te asueguramos que seras un gran docente"
        
        ccc=lower.find('educacion')
        if ccc !=-1:
            return "Sabemos que la educacion debee mejorar cada dia"
        
    env.reset()
    env.load("examen.clp")
    


### Identifica el idioma, y traduce de ser el caso

In [8]:
#CREAR UNA CONDICIÓN PARA QUE LA CONVERSAIÓN SE DETENGA O CONTINUE
condicion=''


# CONVERSASIÓN CON ASSISTANT WATSON
def assistant_bot(pregunta_persona):
    
    message = assistant.message(
        '5d54ad00-095a-4d6f-b630-5bb1bb72fea1',
        session['session_id'],
        input={
            'message_type':'text',
            'text':pregunta_persona
        }
    ).get_result()
    #print(json.dumps(message, indent=2))
    # RECORRO EL JSON RESULT Y OBTENGO EL CAMPO GENERIC QUE ES LA RESPUESTA 
    for key,value in message.items():
        for salida in value["entities"]:
            salida
            
            
        for entityProperty in value["generic"]:
            str(entityProperty)
    
    condicion  = str(salida) 
    res_salida = condicion.split(':',2)
    respuesta = res_salida[1]
    condicion = str(respuesta)
    res_final = condicion.split(',',1)
    condicion =str(res_final[0])
    #print(condicion)
    
    #TEXTO
    response=str(entityProperty)
    res=response.split(':',2)
    paso_valor=str(res[2])
    return paso_valor

#VUELVO A TRADUCIR
def translate_again(texto):
    
    translation = language_translator.translate(
        text = texto, model_id = 'es-en').get_result()
    #print(json.dumps(translation, indent=2, ensure_ascii= False))
    texto=translation['translations']
    transformado=str(texto)
    #print(transformado)
    t=transformado.split(" ", 1)
    #print(t[1])
    v=str(t[1]).split("}",1)
    texto=str(v)
    text_speech_en(texto)      #SE PASA DE TEXTO A VOZ 
    return str(v[0])


# FUNCIÓN DE TRADUCIR 
def translate(texto):
    
    translation = language_translator.translate(
        text = texto, model_id = 'en-es').get_result()
    #print(json.dumps(translation, indent=2, ensure_ascii= False))
    texto=translation['translations']
    transformado=str(texto)
   # print(transformado)
    t=transformado.split(" ", 1)
    #print(t[1])
    v=str(t[1]).split("}",1)
    valor_pasar=str(v[0])
    return valor_pasar

#FUNCIÓN QUE DEVULVE EL TEXTO SIN TRADUCIR
def no_translate(texto):
    
    return texto

#PASAR DE TEXTO A VOZ
def text_speech_en(texto):
    file_path = join( 'resources/output.wav')
    with open(file_path,'wb') as audio_file:
        response = text_to_speech.synthesize(
            texto, accept='audio/wav',
            voice="en-US_AllisonVoice").get_result()
        audio_file.write(response.content)
        
        
        
def text_speech_es(texto):
    file_path = join( 'resources/output.wav')
    with open(file_path,'wb') as audio_file:
        response = text_to_speech.synthesize(
            texto, accept='audio/wav',
            voice="es-LA_SofiaVoice").get_result()
        audio_file.write(response.content)
        
def play_audio():
    pygame.mixer.init()
    pygame.mixer.music.load("resources/output.wav")
    pygame.mixer.music.play()
    pygame.event.wait()



def main():
    print('Escribe algo: ')
    print('Write something: ')
   # print(condicion+" main")
    while str(condicion) != 'despedida':
        texto=input()
        #print(texto)
        language = language_translator.identify(
            texto).get_result()
        value_language=[]
        for key,value in language.items():
            for lenguage_iden in value:
                values=lenguage_iden.values()
                language_list=list(values)
                value_language.append(language_list[0])
        if value_language[0] == 'en':
            #print(translate(texto))
            pregunta_persona=translate(texto)
            #print(assistant_bot(pregunta_persona))
            try:
                texto=assistant_bot(pregunta_persona)
                print(translate_again(texto))
                return(translate_again(texto))
            except:
                #print(assistant_bot(pregunta_persona))
                print('Quizá la pregunta no está bien formulada')
                return (('Maybe the question is not fine formuled'))
            
            #
        elif value_language[0] == 'es':
            #print(no_translate(texto))
            pregunta_persona=no_translate(texto)
            
            try:
                texto=assistant_bot(pregunta_persona)
                print(texto)
                return texto
            except:
                #print(assistant_bot(pregunta_persona))
                print('Quizá la pregunta no está bien formulada')
                return ('Quizá la pregunta no está bien formulada')
            text_speech_es(texto)
#play_audio()
main()

## Recibo el texto y traduzco de español a inglés

In [12]:
from functools import partial
window = Tk()

valor_text1=""
window.title("Chat Bot U.P.S ")
window.geometry('800x800')



tab_control = ttk.Notebook(window)

tab1 = ttk.Frame(tab_control)

tab2 = ttk.Frame(tab_control)

tab_control.add(tab1, text='Usuario')

chat = Text(tab1, height=30, width=30).place(x=250, y=300)
print(type(chat))
S = ttk.Scrollbar(chat)


lbl_information = ttk.Label(tab1, text= "")

tab_control.add(tab2, text='Estudiante')

lbl2 = Label(tab2, text= 'label2')

lbl2.grid(column=0, row=0)

tab_control.pack(expand=1, fill='both')
opcion = IntVar() # Como StrinVar pero en entero
box_value=StringVar()
valor_text=StringVar()
genero_txt=StringVar()
question=StringVar()

def neo_j():  
    lbl_nombre= Label(tab1, text = "Ingresa tu nombre \n Nos ayudara a mejorar nustro servicio")
    gender = ttk.Entry(tab1, textvariable = genero_txt).place(x=50, y=30)
    lbl1 = Label(tab1, text= 'Cuentanos, que te motiva a seguir esta carrera?').place(x=15, y = 30)
    lbl1 = Label(tab1, text= 'Aqui, algunas respuestas anteriores ').place(x=15, y = 45)
    
    valores=list_motivations()    
    box = ttk.Combobox(tab1,textvariable=box_value, values=valores+["Otro"]).place(x=15, y=60)
    
    motivaton_txt = ttk.Entry(tab1, textvariable=valor_text).place(x = 200, y =60)
    
    generos = ['Hombre', 'Mujer']
    
    box_genero= ttk.Combobox(tab1,textvariable=genero_txt, values=generos).place(x=15, y=75)
    
    
    
    b=Button(tab1,text="Guardar", command=get_value, width=20).place(x=250, y=100)
    
    bl1 = Label(tab1, text= 'Has tu pregunta').place(x=90, y = 185)
    ttk.Entry(tab1, textvariable=question).place(x = 90, y =200)
    b_question=Button(tab1,text="?", command=chat_bot, width=20).place(x=250, y=200)

def chat_bot():
    q_cb=question.get()
    print(q_cb + "preunta")
    response=main(q_cb)
    #print(response)
    (chat, response)
    #messagebox.showinfo(message=response, title="Recomendacion")
    
def get_value():
    
    mtv_desc = box_value.get()
    gndr = genero_txt.get()
    
    if mtv_desc =='Otro':
        motivacion =valor_text.get()
        response=expert_system(motivacion)
        messagebox.showinfo(message=response, title="Recomendacion")
        if gndr == 'Hombre':
            merge_hombre(motivacion, 1)
        if gndr == 'Mujer':
            merge_mujer(motivacion, 1)
            
        valores=list_motivations()
        #messagebox.showinfo(message="Mensaje", title="Título")
    else:
        primer=mtv_desc.split('{',1)
        print(primer[0])
        segundo = str(primer[1])
        tercero=segundo.split('}',1)
        cuarto =str(tercero[0])
        motivacion=cuarto
        response=expert_system(motivacion)
        print(gndr)
        values(motivacion, gndr)
        messagebox.showinfo(message=response, title="Recomendacion")
        

#Radiobutton(root, text="Opción 1", variable=opcion, 
      #      value=1, command=selec).pack()
neo_j()
window.mainloop()

<class 'NoneType'>
good morningpreunta
Escribe algo: 
Write something: 
good morning
" 'Welcome to the U.P.S. Basic Education career'
